# Projet Diversité et inclusion en entreprise #

**Problématique : Comment les différents aspects sociaux et professionnels (rémunération, formation, conditions de travail, handicap, absentéisme, etc.) influencent-ils la structure et l’évolution des effectifs d’EDF SA sur une période donnée ?**

*Respect de la PEP-8*

In [33]:
%load_ext pycodestyle_magic
%pycodestyle_on

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [2]:
import pandas as pd

#### 1) Comparaison de l'ecart salarial homme/femme selon l'annee ####

In [75]:
salaire = pd.read_csv("data/promotion.csv", delimiter=";")
salaire = salaire[salaire['Type de contrat']=="Statutaire"]
salaire = salaire[salaire['Indicateur']=='Rémunération mensuelle moyenne brute']
salaire = salaire[(salaire['Collège']=='Cadre') | (salaire['Collège']=='Maitrise') | (salaire['Collège']=='Execution')]
salaire = salaire.drop(columns=['Perimètre juridique', 'Perimètre spatial',
       'Spatial perimeter', 'Indicateur', 'Indicator', 'Type de contrat',
       'Type of contract', 'Employee category', 'Plage M3E',
       'M3E classification', 'Gender', 'Unité', 'Unit',
       'Chapitre du bilan social'])
salaire = salaire.rename(columns={'Valeur': 'Salaire mensuel moyen brut'})
salaire

,Année,Collège,Genre,Salaire mensuel moyen brut
6,2023,Cadre,Femme,7175.0
7,2023,Maitrise,Homme,4990.0
46,2023,Maitrise,Femme,4279.0
47,2023,Execution,Homme,3894.0
48,2023,Execution,Femme,3290.0
60,2023,Cadre,Homme,7842.0
74,2022,Maitrise,Homme,4521.0
99,2022,Cadre,Homme,7353.0
100,2022,Maitrise,Femme,3913.0
101,2022,Execution,Homme,3546.0


#### 2) Corrélation entre formation et promotions ####

Ici, nous allons analyser l'effet des formations (nombre d'heures, nombre d'employés ayant été upgrade) avec une petite comparaison entre hommes et femmes

Graphique 1 : 

In [8]:
promotion = pd.read_csv("data/promotion.csv", delimiter=";")
promotion = promotion[promotion["Indicateur"]=="Promotions dans un collège supérieur"]
promotion = promotion[promotion["Type de contrat"]=="Statutaire"]
drop = ['Perimètre juridique', 'Unité', 'Perimètre spatial',
       'Spatial perimeter', 'Indicator',
       'Type of contract', 'Employee category', 'Plage M3E',
       'M3E classification', 'Gender', 'Unit',
       'Chapitre du bilan social', 'Type de contrat', 'Indicateur']
promotion = promotion.drop(columns=drop)
promotion = promotion.groupby(["Année", "Collège", "Genre"]).sum().reset_index()
promotion = promotion.rename(columns={'Collège': 'Evolution'})
promotion['Collège'] = 'Exécution'
promotion.loc[promotion['Evolution'] == 'Maîtrise vers Cadre', 'Collège'] = 'Maitrise'

In [9]:
formation = pd.read_csv("data/formation.csv", delimiter = ";")
formation = formation[(formation["Collège"] == "Maitrise") | (formation["Collège"] == "Exécution")]
formation = formation[formation["Indicateur"] == "Salariés ayant suivi au moins un stage"]
drop = ['Perimètre juridique', 'Unité', 'Perimètre spatial',
       'Spatial perimeter', 'Indicator',
       'Employee category', 'Plage M3E', 'M3E classification',
       'Gender', 'Unit', 'Chapitre du bilan social']
formation = formation.drop(columns=drop)

In [10]:
df1_2 = pd.merge(promotion, formation, on=['Année','Collège','Genre'], how='inner')
df1_2 = df1_2.rename(columns={'Valeur_x': 'Nombre Evolutions'})
df1_2 = df1_2.rename(columns={'Valeur_y': 'Population formee'})

In [87]:
import pandas as pd
import plotly.express as px

# Transformer les données en format long
df_long = df1_2.melt(
    id_vars=['Année', 'Evolution', 'Genre'], 
    value_vars=['Nombre Evolutions', 'Population formee'], 
    var_name='Type de Valeur', 
    value_name='Valeur'
)

# Créer le graphique
fig = px.line(
    df_long,
    x='Année',
    y='Valeur',
    color='Evolution',  # Couleurs différenciées par évolution
    line_dash='Type de Valeur',  # Style de ligne différencié par type de valeur
    facet_row='Genre',  # Séparer les graphiques pour Femme (haut) et Homme (bas)
    title="Corrélation entre le nombre de personnes formées et les évolutions",
    labels={'Valeur': 'Valeur', 'Année': 'Année', 'Type de Valeur': 'Type'},
    height=800
)

# Définir les options du menu déroulant
dropdown_buttons = [
    dict(label="Exécution vers Maîtrise",
         method="update",
         args=[{"visible": ['Exécution vers Maîtrise' in trace.name for trace in fig.data]},
               {"title": "Corrélation pour l'évolution : Exécution vers Maîtrise"}]),
    dict(label="Maîtrise vers Cadre",
         method="update",
         args=[{"visible": ['Maîtrise vers Cadre' in trace.name for trace in fig.data]},
               {"title": "Corrélation pour l'évolution : Maîtrise vers Cadre"}])
]

# Initialiser les courbes visibles pour "Exécution vers Maîtrise" (par défaut)
default_visible = ['Exécution vers Maîtrise' in trace.name for trace in fig.data]
fig.for_each_trace(lambda trace: trace.update(visible='legendonly'))  # Masquer toutes les traces par défaut
for trace, visible in zip(fig.data, default_visible):
    trace.visible = True if visible else 'legendonly'

# Mettre à jour la mise en page pour inclure le menu déroulant
fig.update_layout(
    updatemenus=[
        dict(
            buttons=dropdown_buttons,
            direction="down",
            showactive=True,
            x=1.35,  # Position horizontale
            xanchor="right",
            y=1.1,  # Position verticale
            yanchor="top"
        )
    ],
    legend_title="Évolution et Type de Valeur"
)

# Afficher le graphique
fig.show()

#### 3) Impact des conditions de travail sur l’absentéisme ET LES DE;ISSIONS####

nuage de points


In [113]:
absence_maladie = pd.read_csv("data/absences.csv", delimiter=";")
absence_maladie = absence_maladie[absence_maladie["Type de contrat"]=="Statutaires"]
drop = ['Perimètre juridique', 'Perimètre spatial',
       'Spatial perimeter', 'Indicator',
       'Type of contract', 'Employee category', 'Gender',
       'Unit', 'Type de contrat', 'Chapitre du bilan social', 'Genre']
absence_maladie = absence_maladie.drop(columns=drop)
absence_maladie = absence_maladie[absence_maladie["Indicateur"] == "Absence pour maladie"]
absence_maladie = absence_maladie.groupby(["Année", "Collège", "Indicateur", "Unité"]).sum().reset_index()
absence_maladie

,Année,Collège,Indicateur,Unité,Valeur
0,2017,Cadre,Absence pour maladie,heures,1081209.0
1,2017,Exécution,Absence pour maladie,heures,756905.0
2,2017,Maîtrise,Absence pour maladie,heures,2330284.0
3,2018,Cadre,Absence pour maladie,heures,1118943.0
4,2018,Exécution,Absence pour maladie,heures,702581.0
5,2018,Maîtrise,Absence pour maladie,heures,2288034.0
6,2019,Cadre,Absence pour maladie,heures,1110863.0
7,2019,Exécution,Absence pour maladie,heures,582413.0
8,2019,Maîtrise,Absence pour maladie,heures,2191619.0
9,2020,Cadre,Absence pour maladie,heures,1073816.0


2:69: E225 missing whitespace around operator
2:80: E501 line too long (84 > 79 characters)
4:8: E128 continuation line under-indented for visual indent
5:8: E128 continuation line under-indented for visual indent
6:8: E128 continuation line under-indented for visual indent
8:80: E501 line too long (90 > 79 characters)
9:80: E501 line too long (106 > 79 characters)


In [83]:
vacances = pd.read_csv("data/absences.csv", delimiter=";")
vacances = vacances[vacances["Type de contrat"]=="Statutaires"]
drop = ['Perimètre juridique', 'Perimètre spatial',
       'Spatial perimeter', 'Indicator',
       'Type of contract', 'Employee category', 'Gender',
       'Unit', 'Type de contrat', 'Chapitre du bilan social', 'Genre']
vacances = vacances.drop(columns=drop)
vacances = vacances[vacances["Indicateur"] == "Absence pour congés autorisés"]
vacances = vacances.groupby(["Année", "Collège", "Indicateur", "Unité"]).sum().reset_index()
vacances = vacances.rename(columns={'Valeur': 'Total des heures de congés'})
vacances = vacances.drop(columns=['Indicateur', 'Unité'])
vacances

,Année,Collège,Total des heures de congés
0,2017,Cadre,327130.0
1,2017,Exécution,71491.0
2,2017,Maîtrise,362252.0
3,2018,Cadre,350507.0
4,2018,Exécution,69055.0
5,2018,Maîtrise,370494.0
6,2019,Cadre,270791.0
7,2019,Exécution,51543.0
8,2019,Maîtrise,311058.0
9,2020,Cadre,235808.0


In [117]:
conditions = pd.read_csv("data/working_conditions.csv", delimiter = ";")
conditions = conditions[conditions["Type de contrat"]=="Statutaires"]
drop = ['Perimètre juridique', 'Perimètre spatial',
       'Spatial perimeter', 'Indicator', 'Type de contrat',
       'Type of contract', 'Time range',
       'Employee category', 'Gender', 'Unit',
       'Chapitre du bilan social']
conditions = conditions.drop(columns=drop)
conditions

,Année,Indicateur,Tranche horaire,Collège,Genre,Valeur,Unité
2,2023,Salariés à temps partiel en décembre,Moins de 20 heures,Maitrise,Homme,32.0,Nombre
5,2023,Salariés à temps partiel en décembre,De 20 heures à moins de 30 heures,Cadre,Homme,54.0,Nombre
6,2023,Salariés à temps partiel en décembre,De 20 heures à moins de 30 heures,Maitrise,Femme,330.0,Nombre
13,2023,Salariés en réduction collective du temps de t...,NaN,Cadre,Femme,91.0,Nombre
14,2023,Salariés en réduction collective du temps de t...,NaN,Maitrise,Femme,632.0,Nombre
...,...,...,...,...,...,...,...
667,2017,Salariés en réduction collective du temps de t...,NaN,Maitrise,Homme,938.0,Nombre
676,2017,Salariés à temps partiel en décembre,De 20 heures à moins de 30 heures,Maitrise,Homme,77.0,Nombre
680,2017,Salariés à temps partiel en décembre,De 30 heures à moins de 35 heures,Maitrise,Homme,189.0,Nombre
682,2017,Salariés en réduction collective du temps de t...,NaN,Cadre,Homme,239.0,Nombre


1:66: E251 unexpected spaces around keyword / parameter equals
1:68: E251 unexpected spaces around keyword / parameter equals
2:54: E225 missing whitespace around operator


4:8: E128 continuation line under-indented for visual indent
5:8: E128 continuation line under-indented for visual indent
6:8: E128 continuation line under-indented for visual indent
7:8: E128 continuation line under-indented for visual indent


In [84]:
salaire = pd.read_csv("data/promotion.csv", delimiter=";")
salaire = salaire[salaire["Indicateur"]=="Rémunération mensuelle moyenne brute"]
drop = ['Perimètre juridique', 'Perimètre spatial',
       'Spatial perimeter', 'Indicator',
       'Type of contract', 'Employee category', 'Plage M3E',
       'M3E classification', 'Gender', 'Unit',
       'Chapitre du bilan social', 'Type de contrat', 'Genre']
salaire = salaire.drop(columns=drop)
salaire = salaire.groupby(["Année", "Collège", "Indicateur", "Unité"]).sum().reset_index()
salaire = salaire.rename(columns={'Valeur': 'Salaire mensuel brut moyen (€)'})
salaire = salaire.drop(columns=['Indicateur', 'Unité'])
salaire

,Année,Collège,Salaire mensuel brut moyen (€)
0,2017,Cadre,41549.0
1,2017,Execution,10345.0
2,2017,Maitrise,17210.0
3,2018,Cadre,41849.0
4,2018,Execution,10351.0
5,2018,Maitrise,17735.0
6,2019,Cadre,40688.0
7,2019,Execution,11302.0
8,2019,Maitrise,17586.0
9,2020,Cadre,39837.0


#### 4) Focus sur les salariés non-statuaires ####

https://ufpi.reference-syndicale.fr/2017/06/stop-aux-cdi-non-statutaires-a-tous-les-etages/

un grqphiaue qvec deux courbes et une liste de choix en fonction

In [104]:
salaire = pd.read_csv("data/promotion.csv", delimiter=";")
drop = ['Perimètre juridique', 'Perimètre spatial',
       'Spatial perimeter', 'Indicator',
       'Type of contract', 'Employee category', 'Plage M3E',
       'M3E classification', 'Gender', 'Unit',
       'Chapitre du bilan social', 'Type de contrat', 'Genre']
salaire = salaire.drop(columns=drop)
salaire = salaire.groupby(["Année", "Collège", "Indicateur", "Unité"]).sum().reset_index()
salaire

,Année,Indicateur,Collège,Valeur,Unité
0,2023,Stagiaires scolaires,Maîtrise,492.0,nombre
1,2023,Salariés détachés en mobilité à l'international,NaN,23.0,nombre
2,2023,Salariés détachés accueillis,NaN,1544.0,nombre
3,2023,Salariés détachés accueillis,NaN,44.0,nombre
4,2023,Stagiaires scolaires,Cadre,6.0,nombre
...,...,...,...,...,...
86,2017,Salariés détachés accueillis,NaN,801.0,nombre
87,2017,Salariés détachés accueillis,NaN,18.0,nombre
88,2017,Stagiaires scolaires,Cadre,3.0,nombre
89,2017,Stagiaires scolaires,Exécution,148.0,nombre


3:8: E128 continuation line under-indented for visual indent
3:61: E231 missing whitespace after ','
4:8: E128 continuation line under-indented for visual indent


In [118]:
absence_full = pd.read_csv("data/absences.csv", delimiter=";")
drop = ['Perimètre juridique', 'Perimètre spatial',
       'Spatial perimeter', 'Indicator',
       'Type of contract', 'Employee category', 'Gender',
       'Unit', 'Type de contrat', 'Chapitre du bilan social', 'Genre']
absence_full = absence_full.drop(columns=drop)
absence_full = absence_full.groupby(["Année", "Collège", "Indicateur", "Unité"]).sum().reset_index()
absence_full

,Année,Collège,Indicateur,Unité,Valeur
0,2017,Cadre,Absence pour congé de maternité ou d'adoption,heures,345432.0
1,2017,Cadre,Absence pour congé de paternité et d'accueil d...,heures,59396.0
2,2017,Cadre,Absence pour congés autorisés,heures,328844.0
3,2017,Cadre,Absence pour maladie,heures,1095570.0
4,2017,Exécution,Absence pour congé de maternité ou d'adoption,heures,106915.0
...,...,...,...,...,...
79,2023,Exécution,Absence pour maladie,heures,395846.0
80,2023,Maîtrise,Absence pour congé de maternité ou d'adoption,heures,159668.0
81,2023,Maîtrise,Absence pour congé de paternité et d'accueil d...,heures,122928.0
82,2023,Maîtrise,Absence pour congés autorisés,heures,278074.0


3:8: E128 continuation line under-indented for visual indent
4:8: E128 continuation line under-indented for visual indent
5:8: E128 continuation line under-indented for visual indent
7:80: E501 line too long (100 > 79 characters)
